In [ ]:
import json
import pickle

In [ ]:
dataset_path = "./data/movie_lens_100k_output_path.json"
with open(dataset_path) as json_file:
    _dataset_path = json.load(json_file)

dataset = {}
with open(_dataset_path["train_users_dict"], "rb") as pkl_file:
    dataset["train_users_dict"] = pickle.load(pkl_file)

with open(_dataset_path["train_users_history_lens"], "rb") as pkl_file:
    dataset["train_users_history_lens"] = pickle.load(pkl_file)

with open(_dataset_path["eval_users_dict"], "rb") as pkl_file:
    dataset["eval_users_dict"] = pickle.load(pkl_file)

with open(_dataset_path["eval_users_history_lens"], "rb") as pkl_file:
    dataset["eval_users_history_lens"] = pickle.load(pkl_file)

with open(_dataset_path["users_history_lens"], "rb") as pkl_file:
    dataset["users_history_lens"] = pickle.load(pkl_file)

# with open(_dataset_path["movies_id_to_movies"], "rb") as pkl_file:
#     dataset["movies_id_to_movies"] = pickle.load(pkl_file)

with open(_dataset_path["movies_groups"], "rb") as pkl_file:
    dataset["movies_groups"] = pickle.load(pkl_file)

In [ ]:
dataset["train_users_history_lens"].sort()
dataset["train_users_history_lens"]

In [ ]:
import plotly.express as px

fig = px.histogram(list(dataset["movies_groups"].values()))
fig.update_layout(
    xaxis_title = "Groups",
    yaxis_title = "Movies",
)
fig.show()

In [ ]:
import pandas as pd

df_rating = pd.read_csv("data/ml-100k/ratings.csv")
df_rating

In [ ]:
users_dict = {user: [] for user in set(df_rating["user_id"])}
users_dict

In [ ]:
ratings_df_gen = df_rating.iterrows()
ratings_df_gen

In [ ]:
users_dict_for_history_len = {
    user: [] for user in set(df_rating["user_id"])
}
users_dict_for_history_len

In [ ]:
for data in ratings_df_gen:
    users_dict[data[1]["user_id"]].append(
        (data[1]["movie_id"], data[1]["rating"])
    )
    if data[1]["rating"] >= 4:
        users_dict_for_history_len[data[1]["user_id"]].append(
            (data[1]["movie_id"], data[1]["rating"])
        )

In [ ]:
users_dict[301]

In [ ]:
users_dict_for_history_len[301]

In [ ]:
users_history_lens = [
    len(users_dict_for_history_len[u])
    for u in set(df_rating["user_id"])
]

In [ ]:
users_history_lens[301]

In [ ]:
x = [
    len(users_dict[u])
    for u in set(df_rating["user_id"])
]

In [ ]:
users_history_lens < x

In [ ]:
user_items = {data[0]: data[1] for data in users_dict[0]}
user_items

In [ ]:
[data[0] for data in users_dict[0][: 5]]

In [ ]:
df_rating[df_rating["user_id"] == 0] #.sort_values("timestamp")

In [ ]:
users_history_lens

In [ ]:
train_users_history_lens = users_history_lens[:train_users_num]
train_users_history_lens

In [ ]:
users_num = max(df_rating["user_id"]) + 1
items_num = max(df_rating["movie_id"]) + 1

# 6041 3953
print(users_num, items_num)

# Training setting
train_users_num = int(users_num * 0.8)
train_items_num = items_num
# train_users_dict = {k: users_dict.get(k) for k in range(1, train_users_num + 1)}
# train_users_history_lens = users_history_lens[:train_users_num]


In [ ]:
x = {1: 15, 2: 7, 3: 4, 4: 10, 5: 12, 6: 7, 7: 8, 8: 6, 9: 10, 10: 15}

In [ ]:
15 + 7 + 4 + 10 + 12 + 7 + 8 + 6 + 10 + 15

In [ ]:
sum(x.values())

In [ ]:
import math
(1 * math.log((1 + (10/100)))) + \
    (1 * math.log((1 + (10/100)))) + \
        (1 * math.log((1 + (10/100)))) + \
            (1 * math.log((1 + (10/100)))) + \
                (1 * math.log((1 + (10/100)))) + \
                    (1 * math.log((1 + (10/100)))) + \
                        (1 * math.log((1 + (10/100)))) + \
                            (1 * math.log((1 + (10/100)))) + \
                                (1 * math.log((1 + (10/100)))) + \
                                    (1 * math.log((1 + (10/100))))

In [ ]:
100000/10

In [ ]:
import math
import numpy as np
(1 * np.log((1 + (6438/100000)))) + \
    (1 * np.log((1 + (33881/100000)))) + \
        (1 * np.log((1 + (13884/100000)))) + \
            (1 * np.log((1 + (24639/100000)))) + \
                (1 * np.log((1 + (10216/100000)))) + \
                    (1 * np.log((1 + (4172/100000)))) + \
                        (1 * np.log((1 + (2862/100000)))) + \
                            (1 * np.log((1 + (1635/100000)))) + \
                                (1 * np.log((1 + (749/100000)))) + \
                                    (1 * np.log((1 + (1524/100000))))

In [ ]:
{5: round(6438/100000, 2),
 1: round(33881/100000, 2),
 3: round(13884/100000, 2),
 2: round(24639/100000, 2),
 4: round(10216/100000, 2),
 6: round(4172/100000, 2),
 7: round(2862/100000, 2),
 8: round(1635/100000, 2),
 10: round(749/100000, 2),
 9: round(1524/100000, 2)
}

In [ ]:
0.1 * math.log((1 + 10/100), 2)

In [ ]:
81-94

In [ ]:
13-9